In [1]:
import spacy
#from spacy_readability import Readability
import json
import tabulate

nlp = spacy.load("de_core_news_sm")

In [2]:
dictionary = {}
dictionary_lemma = {}
dictionary_adj = {}
dictionary_noun = {}
dictionary_verb = {}

In [3]:
def getEmptyElement():
    return {"count": 0, "tp": 0, "tn": 0, "fp": 0, "fn": 0}
def setProp(label, prediction):
    if label and prediction:
        return 'tp'
    if not label and not prediction:
        return 'tn'
    if not label and prediction:
        return 'fp'
    if label and not prediction:
        return 'fn'
def addProp(name, element):
    element['count'] += 1
    element[name] += 1
    return element

def createLibrary(data):
    for d in data:
        prop = setProp( d['label'], d['prediction'])
        for t in d['tokens']:
            if t.text not in dictionary:
                dictionary[t.text] =  getEmptyElement()
            addProp(prop, dictionary[t.text])
            
            if t.lemma_ not in dictionary_lemma:
                dictionary_lemma[t.lemma_] =  getEmptyElement()
            addProp(prop, dictionary_lemma[t.lemma_])
                
            if t.pos_ == "ADJ":
                if t.text not in dictionary_adj:
                    dictionary_adj[t.text] =  getEmptyElement()
                addProp(prop, dictionary_adj[t.text])
                
            if t.pos_ == "NOUN":
                if t.text not in dictionary_noun:
                    dictionary_noun[t.text] =  getEmptyElement()
                addProp(prop, dictionary_noun[t.text])
                
            if t.pos_ == "VERB":
                if t.text not in dictionary_verb:
                    dictionary_verb[t.text] =  getEmptyElement()
                addProp(prop, dictionary_verb[t.text])
        
def load_data(path, createlib = False):
    with open(path) as json_file:
        data = json.load(json_file)
        for comment in data:
            doc = nlp(comment['text'])
            comment['tokens'] = doc
        if createlib:
            createLibrary(data)
        return data

In [4]:
def printFirst100():
    s = sorted(dictionary, key=dictionary.get, reverse=True)
    def lengthFilter(e):
        return len(e)>3 and (len(e) > 5 or e[0].isupper())
    s = list(filter(lengthFilter, s))
    for i in range(100):
        print(s[i])